# ?

In [1]:
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd

In [3]:
from load import *
from torch.utils.data import Dataset, DataLoader
import glob as glob

In [4]:
import numpy as np

In [5]:
train_28049_2_csv = pd.read_csv('./train_28002.csv')

In [6]:
train_28049_2_csv.head()

,img_id,img_path,mask_rle
0,TRAIN_5425_10,./train_img_crop/TRAIN_5425_10.png,247 12 470 13 695 12 717 1 919 12 1143 13 1160...
1,TRAIN_2224_19,./train_img_crop/TRAIN_2224_19.png,66 22 290 22 514 22 738 22 962 22 1186 22 1411...
2,TRAIN_1157_19,./train_img_crop/TRAIN_1157_19.png,92 6 193 28 319 2 417 26 642 23 866 21 1091 19...
3,TRAIN_6761_25,./train_img_crop/TRAIN_6761_25.png,32 40 170 20 256 40 394 20 480 40 618 20 704 4...
4,TRAIN_3521_9,./train_img_crop/TRAIN_3521_9.png,-1


In [7]:
x_path = train_28049_2_csv.img_path
y = train_28049_2_csv.mask_rle

In [ ]:
train_dataset, valid_dataset = train_test_split(dataset, test_size = .1, random_state=42, shuffle=True)
train_dataloader = Dataloader(train_dataset , ...)
valid_dataloader = Dataloader(data_dataset, ...)

In [17]:
x_train, x_val, y_train, y_val = train_test_split(x_path, y, test_size=.1, random_state=42, shuffle=True)

In [ ]:
x_train

# 정환이꺼 ^_^

In [1]:
import os
import cv2
import pandas as pd
import numpy as np

import torch
import torch.optim
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

import segmentation_models_pytorch as smp
from segmentation_models_pytorch.losses import DiceLoss, TverskyLoss, FocalLoss, LovaszLoss

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from run_length_encoding import *
from load import *
from loss import *
from dacon_dice import *

from collections import OrderedDict


In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0,1"  # Set the GPUs 0 and 1 to use


#get gpu DEVICE
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 구성 별로 지정 필요.
SEED = 18 


In [3]:
ARCHITECTURE = 'UnetPlusPlus' # UnetPlusPlus, DeepLabV3, DeepLabV3Plus
ENCODER = 'timm-efficientnet-b7' # resnet152
ENCODER_WEIGHT= 'noisy-student' # 1: imagenet
N_CLASSES = 1
ACTIVATION = None
OPTIMIZER = 'AdamW'
SAVED_MODEL_PATH = '/root/jupyter/Dacon/deeplabv3p/model_save_{}_{}_4/'.format(ARCHITECTURE,ENCODER)

# Train Parameters
TRAIN_DATA_CSV = './train_28002.csv'
BATCH_SIZE = 48 # 2GPUs Maximum
VALID_SET_RATIO = .1
START_EPOCH = 1 # 고정 
NUM_EPOCH = 30
LOSS_PATH = "./loss_history/" # 고정
LEARNING_RATE = 1e-4 
WEIGHT_DECAY = 5.0e-02

INF = float('inf') # 고정
tol = 1e-6 



In [4]:
if not os.path.exists(SAVED_MODEL_PATH):
    print('create directory {}'.format(SAVED_MODEL_PATH))
    os.mkdir(SAVED_MODEL_PATH)


create directory /root/jupyter/Dacon/deeplabv3p/model_save_UnetPlusPlus_timm-efficientnet-b7_4/


In [5]:
is_weight = get_weight(SAVED_MODEL_PATH)
if is_weight == False:
    print('there is no saved model')
    model = get_model(ARCHITECTURE)
    model = model(classes=N_CLASSES,
                encoder_name=ENCODER,
                encoder_weights=ENCODER_WEIGHT,
                activation=ACTIVATION)
    
    model = nn.DataParallel(model) 
    model.to(DEVICE)
else:
    last_epoch, last_ckpt = is_weight
    print('last epoch is {}'.format(last_epoch))
    print('model-{} loaded..'.format(last_epoch))
    model = get_model(ARCHITECTURE)
    model = model(classes=N_CLASSES,
                encoder_name=ENCODER,
                encoder_weights=ENCODER_WEIGHT,
                activation=ACTIVATION)
    
    model = nn.DataParallel(model) 
    model.load_state_dict(last_ckpt, strict=False)
    model.to(DEVICE)
    START_EPOCH = last_epoch+1


there is no saved model


In [6]:
optimizer = get_optimizer(OPTIMIZER)
optimizer = optimizer(model.parameters(),lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)


# Transform
transform_train = get_transform_for_train(version=4) # version4 fix
transform_valid = get_transform_for_test()

train_dataloader, validation_dataloader = train_valid_seed(csv_file=TRAIN_DATA_CSV,
                                                      transform_train=transform_train,
                                                      transform_valid=transform_valid,
                                                      batch_size=BATCH_SIZE,
                                                      test_size=VALID_SET_RATIO,
                                                     random_seed=SEED,
                                                     shuffle=True)

In [12]:
bceLoss = torch.nn.BCEWithLogitsLoss()
dice_loss = DiceLoss(mode='binary')

pasted_epoch_score = [INF] 
pasted_dice_loss = []
pasted_bce_loss = []
pasted_loss = []

In [13]:


for epoch in range(START_EPOCH, START_EPOCH+NUM_EPOCH):
    
    model.train()

    epoch_bce_loss = 0
    epoch_dice_loss = 0
    epoch_train_loss = 0
    
    for imgs, msks in tqdm(train_dataloader):
        imgs = imgs.to(device=DEVICE, dtype = torch.float)
        msks = msks.to(device=DEVICE, dtype = torch.float)
        
        optimizer.zero_grad()
        outputs = model(imgs)
        
        
        #diceloss = dice_loss(outputs, msks.unsqueeze(1))
        bceloss = bceLoss(outputs, msks.unsqueeze(1))
        diceloss = dice_loss(outputs, msks.unsqueeze(1))
        
        w1 = .7
        w2 = .4
        loss = w1 * bceloss + w2 * diceloss
        
        loss.mean().backward()
        optimizer.step()
        
        epoch_bce_loss += bceloss.item()
        epoch_dice_loss += diceloss.item()
        epoch_train_loss += loss.item()
        
    pasted_bce_loss.append(epoch_bce_loss/len(train_dataloader))
    pasted_dice_loss.append(epoch_dice_loss/len(train_dataloader))
    pasted_loss.append(epoch_train_loss/len(train_dataloader))
    
    epoch_score = 0
    #val_asl_loss = 0
    
    with torch.no_grad():
        model.eval()
        result = []
        for imgs,msks in tqdm(validation_dataloader):
            imgs = imgs.to(device=DEVICE, dtype = torch.float)
            msks = msks.to(device=DEVICE, dtype = torch.float)
            outputs = model(imgs)
            
            dc_sc = calculate_dice_scores_from_rle(outputs,msks)
            epoch_score += dc_sc
            
            
            #val_asl = validation_asl(outputs, msks.unsqueeze(1))
            #val_asl_loss += val_asl.item()
    
    
    print(f'Epoch {epoch}')
    print(f'BCE Loss: {epoch_bce_loss/len(train_dataloader)}')
    print(f'DICE Loss: {epoch_dice_loss/len(train_dataloader)}')
    print(f'Total Loss: {epoch_train_loss/len(train_dataloader)}')
    
    #print(f'Validation Asymmetric Loss: {val_asl_loss/len(validation_dataloader)}')
    print(f'Validation Dice Score: {epoch_score/len(validation_dataloader)}')
    
    pasted_epoch_score.append(epoch_score/len(validation_dataloader))
    #pasted_val_loss.append(val_asl_loss/len(validation_dataloader))
    
    # save a weight every epoch
    path = SAVED_MODEL_PATH + '{}_{}-{num:0004d}.pth'.format(ARCHITECTURE,ENCODER,num=epoch)
    torch.save(model.state_dict(), path)
    
    if np.abs(pasted_epoch_score[-2] - pasted_epoch_score[-1])< tol:
        print('Early Stop')
        break;
    
# save epoch losses as .csv
loss_n_score = [pasted_bce_loss, pasted_dice_loss, pasted_epoch_score[1:]]
loss_df = pd.DataFrame(loss_n_score)
loss_df = loss_df.transpose()
# !!!!파일명 변경 필요!!!!
loss_df.to_csv(path_or_buf=LOSS_PATH + '{}-{}_4'.format(ARCHITECTURE, ENCODER)+'.csv', 
               index=False,
               header=['train_BCELoss','train_DiceLoss','val_DiceScore'])


100% 526/526 [05:35<00:00,  1.57it/s]
 15% 9/59 [00:10<00:55,  1.12s/it]


KeyboardInterrupt: 

In [ ]:
'''
*** 변경사항 ***
SAVED_MODEL_PATH -> model_save_{ARCHITECTURE}_{ENCODER}/ 로 자동 지정 및 자동 생성.
SEED -> architecture, encoder등의 변경 사항이 있을 경우 변경 필요.
        모든 파라미터가 동일한 경우에 대해 이어서 학습 할 경우 동일한 SEED사용.
get_transform_for_train(), get_transform_for_test() -> load.py
get_dataset(), get_dataloader(),random_split_train_valid() -> load.train_valid_seed()로 변경.

*** 주의 사항 ***
모델이 같아도, 다른 paramters 변경 할 경우 이전 모델 학습 이어서 불가.
SAVED_MODEL_PATH 따로 지정 해주어야 함.
아래 쪽에 csv파일 저장되는 부분이랑 model weight저장되는 부분 한번 더 확인하고 실행.

*** 사용 방법 ***
해당 파일을 copy and paste.
파일 명 변경 -> train_ARCHITECTURE_ENCODER.py
model params들을 변경해준다.
SEED 변경해준다.
 

'''

import os
import cv2
import pandas as pd
import numpy as np

import torch
import torch.optim
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

import segmentation_models_pytorch as smp
from segmentation_models_pytorch.losses import DiceLoss, TverskyLoss, FocalLoss, LovaszLoss

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from run_length_encoding import *
from load import *
from loss import *
from dacon_dice import *

from collections import OrderedDict



#gpu setting
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0,1"  # Set the GPUs 0 and 1 to use


#get gpu DEVICE
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 구성 별로 지정 필요.
SEED = 18 

# Model Parameters
ARCHITECTURE = 'UnetPlusPlus' # UnetPlusPlus, DeepLabV3, DeepLabV3Plus
ENCODER = 'timm-efficientnet-b7' # resnet152
ENCODER_WEIGHT= 'noisy-student' # 1: imagenet
N_CLASSES = 1
ACTIVATION = None
OPTIMIZER = 'AdamW'
SAVED_MODEL_PATH = '/root/jupyter/Dacon/deeplabv3p/model_save_{}_{}_3/'.format(ARCHITECTURE,ENCODER)

# Train Parameters
TRAIN_DATA_CSV = './train_28002.csv'
BATCH_SIZE = 48 # 2GPUs Maximum
VALID_SET_RATIO = .1
START_EPOCH = 1 # 고정 
NUM_EPOCH = 30
LOSS_PATH = "./loss_history/" # 고정
LEARNING_RATE = 1e-4 
WEIGHT_DECAY = 5.0e-02

# ASL loss
GAMMA_NEG=3
GAMMA_POS=1

# Others
INF = float('inf') # 고정
tol = 1e-6 


# 모델 저장 경로(SAVED_MODEL_PATH) 확인 및 디렉토리 생성
if not os.path.exists(SAVED_MODEL_PATH):
    print('create directory {}'.format(SAVED_MODEL_PATH))
    os.mkdir(SAVED_MODEL_PATH)
    


#load weight
is_weight = get_weight(SAVED_MODEL_PATH)
if is_weight == False:
    print('there is no saved model')
    model = get_model(ARCHITECTURE)
    model = model(classes=N_CLASSES,
                encoder_name=ENCODER,
                encoder_weights=ENCODER_WEIGHT,
                activation=ACTIVATION)
    
    model = nn.DataParallel(model) 
    model.to(DEVICE)
else:
    last_epoch, last_ckpt = is_weight
    print('last epoch is {}'.format(last_epoch))
    print('model-{} loaded..'.format(last_epoch))
    model = get_model(ARCHITECTURE)
    model = model(classes=N_CLASSES,
                encoder_name=ENCODER,
                encoder_weights=ENCODER_WEIGHT,
                activation=ACTIVATION)
    
    model = nn.DataParallel(model) 
    model.load_state_dict(last_ckpt, strict=False)
    model.to(DEVICE)
    START_EPOCH = last_epoch+1

    
                                                 
# OPTIMIZER 
optimizer = get_optimizer(OPTIMIZER)
optimizer = optimizer(model.parameters(),lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)


# Transform
transform_train = get_transform_for_train(version=4) # version4 fix
transform_valid = get_transform_for_test()

train_dataloader, validation_dataloader = train_valid_seed(csv_file=TRAIN_DATA_CSV,
                                                      transform_train=transform_train,
                                                      transform_valid=transform_valid,
                                                      batch_size=BATCH_SIZE,
                                                      test_size=VALID_SET_RATIO,
                                                     random_seed=SEED,
                                                     shuffle=True)


# LOSS
#bceLoss = torch.nn.BCEWithLogitsLoss()
#dice_loss = DiceLoss(mode='binary')
#iouloss = IoULoss()
asl_loss = AsymmetricLoss(gamma_neg=GAMMA_NEG, gamma_pos=GAMMA_POS, clip=0.05, disable_torch_grad_focal_loss=True)
#lovasz
tversky_loss = TverskyLoss(mode='binary',alpha=0.7, beta=0.3)



# LISTs of Train Losses and Valid Score
pasted_epoch_score = [INF] 
pasted_val_loss = []
pasted_tversky_loss = []
pasted_asl_loss = []
pasted_loss = []


for epoch in range(START_EPOCH, START_EPOCH+NUM_EPOCH):
    
    model.train()

    epoch_asl_loss = 0
    epoch_tversky_loss = 0
    epoch_train_loss = 0
    
    for imgs, msks in tqdm(train_dataloader):
        imgs = imgs.to(device=DEVICE, dtype = torch.float)
        msks = msks.to(device=DEVICE, dtype = torch.float)
        
        optimizer.zero_grad()
        outputs = model(imgs)
        
        
        #diceloss = dice_loss(outputs, msks.unsqueeze(1))
        tverskyloss = tversky_loss(outputs, msks.unsqueeze(1))
        aslloss = asl_loss(outputs, msks.unsqueeze(1))
        
        loss = aslloss + tverskyloss
        
        loss.mean().backward()
        optimizer.step()
        
        epoch_asl_loss += aslloss.item()
        epoch_tversky_loss += tverskyloss.item()
        epoch_train_loss += loss.item()
        
    pasted_asl_loss.append(epoch_asl_loss/len(train_dataloader))
    pasted_tversky_loss.append(epoch_tversky_loss/len(train_dataloader))
    pasted_loss.append(epoch_train_loss/len(train_dataloader))
    
    epoch_score = 0
    val_asl_loss = 0
    
    with torch.no_grad():
        model.eval()
        result = []
        for imgs,msks in tqdm(validation_dataloader):
            imgs = imgs.to(device=DEVICE, dtype = torch.float)
            msks = msks.to(device=DEVICE, dtype = torch.float)
            outputs = model(imgs)
            
            dc_sc = calculate_dice_scores_from_rle(outputs,msks)
            epoch_score += dc_sc
            
            
            val_asl = validation_asl(outputs, msks.unsqueeze(1))
            val_asl_loss += val_asl.item()
    
    
    print(f'Epoch {epoch}')
    print(f'Asymmetric Loss: {epoch_asl_loss/len(train_dataloader)}')
    print(f'Tversky Loss: {epoch_tversky_loss/len(train_dataloader)}')
    print(f'Total Loss: {epoch_train_loss/len(train_dataloader)}')
    
    print(f'Validation Asymmetric Loss: {val_asl_loss/len(validation_dataloader)}')
    print(f'Validation Dice Score: {epoch_score/len(validation_dataloader)}')
    
    pasted_epoch_score.append(epoch_score/len(validation_dataloader))
    pasted_val_loss.append(val_asl_loss/len(validation_dataloader))
    
    # save a weight every epoch
    path = SAVED_MODEL_PATH + '{}_{}-{num:0004d}.pth'.format(ARCHITECTURE,ENCODER,num=epoch)
    torch.save(model.state_dict(), path)
    
    if np.abs(pasted_epoch_score[-2] - pasted_epoch_score[-1])< tol:
        print('Early Stop')
        break;
    
# save epoch losses as .csv
loss_n_score = [pasted_asl_loss, pasted_dice_loss, pasted_loss, pasted_val_loss, pasted_epoch_score[1:]]
loss_df = pd.DataFrame(loss_n_score)
loss_df = loss_df.transpose()
# !!!!파일명 변경 필요!!!!
loss_df.to_csv(path_or_buf=LOSS_PATH + '{}-{}_3'.format(ARCHITECTURE, ENCODER)+'.csv', 
               index=False,
               header=['train ASLoss','train TverskyLoss','train TotalLoss','val ASLoss','val_DiceScore'])



In [1]:
import pandas as pd

In [2]:
a = pd.read_csv('train_52857.csv')

In [10]:
b = a.drop(columns='Unnamed: 0').reset_index(drop = True).sample(frac= 1)

In [11]:
b

,img_id,img_path,mask_rle
42701,TRAIN_0234_20,./train_img_crop/TRAIN_0234_20.png,8 121 191 34 232 121 415 34 456 121 639 34 680...
21002,TRAIN_6778_5,./train_img_crop/TRAIN_6778_5.png,19948 1 19956 1 20171 6 20395 9 20619 12 20842...
3327,TRAIN_0257_13,./train_img_crop/TRAIN_0257_13.png,17 19 241 19 465 19 686 1 689 19 913 19 1137 1...
37641,TRAIN_5681_17,./train_img_crop/TRAIN_5681_17.png,212 13 436 13 660 13 884 13 1108 13 1322 23 15...
5283,TRAIN_0682_12,./train_img_crop/TRAIN_0682_12.png,158 38 206 19 382 38 430 19 606 38 654 19 830 ...
...,...,...,...
22225,TRAIN_3503_19,./train_img_crop/TRAIN_3503_19.png,162 34 386 35 609 36 833 35 1058 34 1284 32 15...
15995,TRAIN_6454_6,./train_img_crop/TRAIN_6454_6.png,217 8 442 7 666 7 890 7 1115 6 1339 6 1564 5 1...
42505,TRAIN_5276_17,./train_img_crop/TRAIN_5276_17.png,12852 40 13076 40 13300 40 13524 40 13748 40 1...
167,TRAIN_3167_15,./train_img_crop/TRAIN_3167_15.png,12444 37 12668 37 12892 37 13116 37 13340 37 1...


In [12]:
b.to_csv('train_52857.csv', index = False)